In [3]:
%load_ext autoreload
%autoreload 2

import pickle

import click
import screed

from Bio.Blast import NCBIWWW

from khtools.bloom_filter import load_nodegraph, make_peptide_set
from khtools.extract_coding import score_single_translation, encode_peptide, kmerize

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
cd ~/data_lg/kmer-hashing/brawand2011/kmermaid/results_dayhoff11_extract_coding_jaccard_threshold05/extract_coding

/mnt/ibm_lg/olga/kmer-hashing/brawand2011/kmermaid/results_dayhoff11_extract_coding_jaccard_threshold05/extract_coding


In [5]:
ls -lha *mdo*coding_reads_peptides.fasta

-rw-r--r-- 1 olga olga 123M Nov  4 19:54 SRR306752_GSM752603_mdo_kd_M_1_trimmed__coding_reads_peptides.fasta
-rw-r--r-- 1 olga olga 1.5G Nov  4 13:39 SRR306753_GSM752606_mdo_lv_F_1_trimmed__coding_reads_peptides.fasta
-rw-r--r-- 1 olga olga 1.1G Nov  4 17:58 SRR306754_GSM752608_mdo_lv_M_1_trimmed__coding_reads_peptides.fasta
-rw-r--r-- 1 olga olga 1.5G Nov  4 22:24 SRR306755_GSM752611_mdo_ts_M_1_trimmed__coding_reads_peptides.fasta
-rw-r--r-- 1 olga olga 3.2G Nov  4 15:50 SRR306756_GSM752613_mdo_ts_M_2_trimmed__coding_reads_peptides.fasta


In [6]:
! head SRR306752_GSM752603_mdo_kd_M_1_trimmed__coding_reads_peptides.fasta > test.fasta
! cat test.fasta

>SRR306752.27035 Ibis_Run090629:7:1:313:423/1 translation_frame: 3
IAILTGDFPSLEP
>SRR306752.27035 Ibis_Run090629:7:1:313:423/1 translation_frame: -1
WFKGWKVTRKDGN
>SRR306752.27035 Ibis_Run090629:7:1:313:423/1 translation_frame: -2
GSRDGKSPVRMAM
>SRR306752.32778 Ibis_Run090629:7:1:381:125/1 translation_frame: 3
LALSSINTTVSTITFSILFLLT
>SRR306752.32778 Ibis_Run090629:7:1:381:125/1 translation_frame: -2
VRRNRIENVMVETVVLIDDRAK


The NCBI Blast takes hella long so skip this step

In [7]:
# fasta_string = '''>SRR306752.27035 Ibis_Run090629:7:1:313:423/1 translation_frame: 3
# IAILTGDFPSLEP
# >SRR306752.27035 Ibis_Run090629:7:1:313:423/1 translation_frame: -1
# WFKGWKVTRKDGN
# >SRR306752.27035 Ibis_Run090629:7:1:313:423/1 translation_frame: -2
# GSRDGKSPVRMAM
# >SRR306752.32778 Ibis_Run090629:7:1:381:125/1 translation_frame: 3
# LALSSINTTVSTITFSILFLLT
# >SRR306752.32778 Ibis_Run090629:7:1:381:125/1 translation_frame: -2
# VRRNRIENVMVETVVLIDDRAK'''
# result_handle = NCBIWWW.qblast("tblastn", "nt", fasta_string)

In [9]:
csv = 'SRR306752_GSM752603_mdo_kd_M_1_trimmed__coding_scores.csv'

In [10]:
! head $csv

read_id,jaccard_in_peptide_db,n_kmers,classification,filename
SRR306752.24755 Ibis_Run090629:7:1:286:693/1,,,All translations shorter than peptide k-mer size + 1,SRR306752_GSM752603_mdo_kd_M_1_trimmed.fq.gz
SRR306752.26212 Ibis_Run090629:7:1:304:224/1,,,All translations shorter than peptide k-mer size + 1,SRR306752_GSM752603_mdo_kd_M_1_trimmed.fq.gz
SRR306752.27035 Ibis_Run090629:7:1:313:423/1,1.0,3.0,Coding,SRR306752_GSM752603_mdo_kd_M_1_trimmed.fq.gz
SRR306752.31741 Ibis_Run090629:7:1:369:831/1,,,All translations shorter than peptide k-mer size + 1,SRR306752_GSM752603_mdo_kd_M_1_trimmed.fq.gz
SRR306752.32553 Ibis_Run090629:7:1:378:170/1,0.1,10.0,Non-coding,SRR306752_GSM752603_mdo_kd_M_1_trimmed.fq.gz
SRR306752.32778 Ibis_Run090629:7:1:381:125/1,0.8333333333333334,12.0,Coding,SRR306752_GSM752603_mdo_kd_M_1_trimmed.fq.gz
SRR306752.33021 Ibis_Run090629:7:1:384:223/1,0.8333333333333334,12.0,Coding,SRR306752_GSM752603_mdo_kd_M_1_trimmed.fq.gz
SRR306752.33073 Ibis_Run090629:7:1:385:523/1,0

In [11]:
read_id = 'SRR306752.27035 Ibis_Run090629:7:1:313:423/1'
! grep "$read_id" $csv

SRR306752.27035 Ibis_Run090629:7:1:313:423/1,1.0,3.0,Coding,SRR306752_GSM752603_mdo_kd_M_1_trimmed.fq.gz


In [12]:
single_end_folder = '/home/olga/data_lg/kmer-hashing/brawand2011/nfcore-rnaseq-single-end/allspecies/trim_galore'

In [13]:
! zgrep -A 3 "$read_id" $single_end_folder/SRR306752_GSM752603_mdo_kd_M_1_trimmed*

@SRR306752.27035 Ibis_Run090629:7:1:313:423/1
GCATTGCCATCCTTACGGGTGACTTTCCATCCCTTGAACCA
+
44867167967675;84.2439987584<45666423;779


In [14]:
outdir = '/home/olga/data_lg/kmer-hashing/brawand2011/kmermaid/results_dayhoff11_extract_coding_jaccard_threshold05'
peptide_bloom_filter = load_nodegraph(f"{outdir}/bloom_filter/Homo_sapiens__molecule-dayhoff_ksize-11.bloomfilter")

In [30]:
molecule = 'dayhoff'
peptide_ksize = 12

# score_single_translation('IAILTGDFPSLEP', peptide_bloom_filter, peptide_ksize=peptide_ksize, molecule=molecule)

In [31]:

# score_single_translation('WFKGWKVTRKDGN', peptide_bloom_filter, peptide_ksize=peptide_ksize, molecule=molecule)

In [32]:
# score_single_translation('GSRDGKSPVRMAM', peptide_bloom_filter, peptide_ksize=peptide_ksize, molecule=molecule)

In [33]:
peptide_fasta = '/home/olga/data_lg/czbiohub-reference/ensembl/release-97/fasta/homo_sapiens/pep/Homo_sapiens.GRCh38.pep.all.fa.gz'

all_peptide_kmers = make_peptide_set(peptide_fasta, peptide_ksize, molecule)
len(all_peptide_kmers)

110048it [00:17, 6118.52it/s]


9540952

In [34]:
def score_single_translation_from_set(translation,
                             peptide_kmer_set,
                             peptide_ksize,
                             molecule='protein',
                             verbose=True):
    encoded = encode_peptide(translation, molecule)
    kmers = kmerize(str(encoded), peptide_ksize)
    kmers_in_peptide_db = peptide_kmer_set.intersection(kmers)
#     import pdb ; pdb.set_trace()
    n_kmers_in_peptide_db = len(kmers_in_peptide_db)
    
    n_kmers = len(kmers)

    if verbose > 1:
        print(f"\ttranslation: \t{encoded}")
        print("\tkmers:", ' '.join(kmers))

    if verbose > 1:
        # Print keys (kmers) only
        print(f"\tK-mers in peptide database:")
        print(kmers_in_peptide_db)

    fraction_in_peptide_db = n_kmers_in_peptide_db / n_kmers

    return fraction_in_peptide_db, n_kmers


for record in screed.open('test.fasta'):
    name = record['name']
    print(f'\n--- {name} ---')
    seq = record['sequence']
    print(seq)
    fraction_in_peptide_db, n_kmers = score_single_translation_from_set(seq, all_peptide_kmers, 
                                                                      peptide_ksize=peptide_ksize,
                                                                      molecule=molecule, verbose=2)
    print(f'fraction_in_peptide_db: {fraction_in_peptide_db}')
    print(f'n_kmers: {n_kmers}')


--- SRR306752.27035 Ibis_Run090629:7:1:313:423/1 translation_frame: 3 ---
IAILTGDFPSLEP
	translation: 	ebeebbcfbbecb
	kmers: ebeebbcfbbec beebbcfbbecb
	K-mers in peptide database:
{'beebbcfbbecb'}
fraction_in_peptide_db: 0.5
n_kmers: 2

--- SRR306752.27035 Ibis_Run090629:7:1:313:423/1 translation_frame: -1 ---
WFKGWKVTRKDGN
	translation: 	ffdbfdebddcbc
	kmers: ffdbfdebddcb fdbfdebddcbc
	K-mers in peptide database:
{'ffdbfdebddcb', 'fdbfdebddcbc'}
fraction_in_peptide_db: 1.0
n_kmers: 2

--- SRR306752.27035 Ibis_Run090629:7:1:313:423/1 translation_frame: -2 ---
GSRDGKSPVRMAM
	translation: 	bbdcbdbbedebe
	kmers: bbdcbdbbedeb bdcbdbbedebe
	K-mers in peptide database:
{'bbdcbdbbedeb', 'bdcbdbbedebe'}
fraction_in_peptide_db: 1.0
n_kmers: 2

--- SRR306752.32778 Ibis_Run090629:7:1:381:125/1 translation_frame: 3 ---
LALSSINTTVSTITFSILFLLT
	translation: 	ebebbecbbebbebfbeefeeb
	kmers: bebbecbbebbe bebbebfbeefe bbebfbeefeeb becbbebbebfb cbbebbebfbee ebebbecbbebb ecbbebbebfbe ebbecbbebbeb bbebbeb

In [26]:
6**11

362797056

In [27]:
len(all_peptide_kmers)

7850417

In [28]:
100*len(all_peptide_kmers)/6**11

2.1638590694627906